In [ ]:
from networkx import maximum_flow
from networkx import gnm_random_graph
from networkx import draw 
from networkx import adjacency_matrix
from networkx import all_simple_paths
from networkx import shortest_path
from networkx import set_edge_attributes
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from copy import deepcopy

# Graph

In [ ]:
# Density is the proportion of edges in a full connected graph (dense graph has n(n-1) edges where n is n_nodes)
# maximum_flow from networkx has a linear trend wrt density.
# Column generation has an exponential trend wrt density (keep a density < 0.3 !!!)

min_weight = 5
max_weight = 15
n_nodes = 5 * 10 ** 3
n_edges = 249_950 
density = n_edges / (n_nodes * (n_nodes - 1))
print ("density = ", density)
source, destination = 0, 1 # don't need to randomize (graph is randomized)

## Creates a random valuated graph

In [ ]:
def modify_weights_graph (Graph, ctr_lists, new_weights) :
    """ utility function for setting weights (dual problem of shortest path) """
    set_edge_attributes(Graph, 0, name="weight")

    set_edge_attributes(
        Graph,
        values = dict({edge : dual for edge, dual in zip (ctr_lists, new_weights)}),
        name = 'weight'
    )

In [ ]:
Graph = gnm_random_graph(n_nodes, n_edges, directed=True)
Graph_dual = deepcopy(Graph)

for (u, v) in Graph.edges():
    Graph.edges[u,v]['weight'] = np.random.uniform(min_weight, max_weight)

In [ ]:
Adj_matrix = adjacency_matrix(Graph)
Adj_matrix.shape

In [ ]:
%%time
flow, flow_dict = maximum_flow(Graph, _s = source, _t = destination, capacity='weight')

# total flow, flow first node
# flow, flow_dict[0]

Maximum flow is equivalent to maximum path generation **if we consider all possible paths**

# Column Generation

In [ ]:
import pulp as pl

In [ ]:
solver = pl.getSolver('CPLEX_CMD', timeLimit=10)
solver.path

In [ ]:
def get_n_paths (Graph, source, destination, max_iter = 16):
    """ Get the max_iter paths of all_simple_paths generator: Graph: source --|> destination """
    list_ = []
    for (index, path) in enumerate(all_simple_paths(Graph, source, destination, cutoff=10)):
        list_.append(path)
        if (index >= max_iter - 1):
            return (list_)
    return (list_)

In [ ]:
# affichage de n chemins 
len(get_n_paths(Graph, source, destination, max_iter=50))

## Master

In [ ]:
# Etape 0 INIT THE MASTER PROBLEM

lpProb = pl.LpProblem("path_generation", pl.LpMaximize)
lpProb += 0 # initiate the objective function to 0 (necessary line of code)

In [ ]:
# useful to know the final paths at the end and calculate the shortest path for dual problem
path2idx = {}
ctr_lists = []
path_count = 0

In [ ]:
def add_paths (lpProb, paths, path_count, path2idx):
    """ Add paths in lpPoblem (new variables Xp and new constraints or terms in constraints Ce) """

    # pour chaque path ajouter une variable et ses coef. dans les contraintes
    for path in paths :

        # add a var path
        prov = pl.LpVariable("X"+str(path_count), 0, max_weight, pl.LpContinuous)
        lpProb.objective.addterm(prov, 1)
        
        # for each edge on the path, search the associated constraint (indexed by edge)
        for index in range(len(path)-1) :
            index_ctr = str((path[index], path[index + 1]))
            if index_ctr in lpProb.constraints:
                lpProb.constraints[index_ctr].addterm(prov, 1) # add term in constraint
            else:
                lpProb.constraints[index_ctr] = prov <= Graph.edges[(path[index], path[index + 1])]
                ctr_lists.append((path[index], path[index + 1]))
        
        path2idx[path_count] = str(path)
        path_count += 1
    return (path_count, path2idx)

In [ ]:
path_count, path2idx = add_paths (lpProb, get_n_paths (Graph, source, destination, max_iter = 1), path_count, path2idx)

In [ ]:
# solver = pl.CPLEX_PY()
result = lpProb.solve()
# result = lpProb.solve(pl.GUROBI_CMD(options=[("MIPgap", 0.9)]))

## Dual variables

In [ ]:
def get_duals(lpPb):
    """ Get the dual variables from a problem Pulp """
    return [c.pi for _, c in list(lpPb.constraints.items())]

## Iterative procedure

In [ ]:
%%time

go_on = True

while go_on :
    # search of path q (dual problem)
    duals = get_duals(lpProb)
    modify_weights_graph (Graph_dual, ctr_lists, duals)
    path_q = shortest_path(Graph_dual, source, destination, weight="weight")

    # compute reduced cost for dual
    reduced_cost = 1
    for e in range(len(path_q)-1) :
        try :
            reduced_cost -= lpProb.constraints[str((path_q[e], path_q[e + 1]))].pi
        except KeyError : # <|-- constraint does not exist (so dual == 0)
            pass

    # stop procedure ?
    if reduced_cost > 0 : # add path q in lp Problem
        path_count, path2idx = add_paths(lpProb, [path_q], path_count, path2idx)
        lpProb.solve()
    else :
        go_on = False

In [ ]:
for i in range(1) :
    print (i)

In [ ]:
flow = 0
for var in lpProb.variables():
    flow += var.value()
print(flow)

In [ ]:
paths = {path2idx[int(str(var)[1:])] : var.value() for var in lpProb.variables() if var.value() > 0}
paths